In [1]:
import re
import pandas as pd
from pathlib import Path

In [7]:
# ===========================================
# 1. Ler o código-fonte
# ===========================================
file_path = Path("example.py")
code = file_path.read_text()

In [13]:
# ===========================================
# 2. Capturar tabelas lidas e escritas
# ===========================================
reads = re.findall(r'read\.csv\("([^"]+)"', code)
writes = re.findall(r'option\(\s*["\']dbtable["\']\s*,\s*["\']([^"\']+)["\']', code)

In [16]:
# ===========================================
# 3. Capturar criação e transformação de DataFrames
# ===========================================
df_creations = re.findall(r'(\w+_df)\s*=\s*(.+)', code)


In [20]:
# Lista para mapping
mapping_rows = []


# ===========================================
# 4. Analisar os dataframes
# ===========================================
for df_name, expression in df_creations:
    # Identifica origem
    origem = None
    if ".read" in expression:
        origem = re.findall(r'"([^"]+)"', expression)
        origem = origem[0] if origem else None
    else:
        origem = re.findall(r'(\w+_df)', expression)
        origem = ", ".join(set(origem) - {df_name}) or "—"
    # Identifica transformações
    if "join" in expression:
        transformacao = "join"
    elif "filter" in expression:
        transformacao = "filter"
    elif "withColumn" in expression:
        transformacao = "withColumn"
    elif "select" in expression:
        transformacao = "select"
    elif "groupBy" in expression:
        transformacao = "groupBy"
    else:
        transformacao = "—"

    mapping_rows.append({
        "DataFrame": df_name,
        "Origem": origem,
        "Transformação": transformacao
    })

In [22]:

# ===========================================
# 5. Capturar destino final
# ===========================================
destino_final = writes[0] if writes else "—"


In [24]:

# ===========================================
# 6. Criar DataFrame de Mapping
# ===========================================
df_mapping = pd.DataFrame(mapping_rows)
df_mapping["Tabela Final"] = destino_final


In [26]:

# Reorganiza colunas
df_mapping = df_mapping[["Tabela Final", "DataFrame", "Origem", "Transformação"]]


In [ ]:
# ===========================================
# 7. Salvar o mapeamento em CSV/Excel
# ===========================================
df_mapping.to_csv("data_mapping.csv", index=False)


In [ ]:
df_mapping

,Tabela Final,DataFrame,Origem,Transformação
0,vw_vendas_cidade,clientes_df,data/clientes.csv,—
1,vw_vendas_cidade,enderecos_df,data/enderecos.csv,—
2,vw_vendas_cidade,pedidos_df,data/pedidos.csv,—
3,vw_vendas_cidade,itens_df,data/itens_pedido.csv,—
4,vw_vendas_cidade,produtos_df,data/produtos.csv,—
5,vw_vendas_cidade,clientes_df,—,withColumn
6,vw_vendas_cidade,enderecos_df,—,withColumn
7,vw_vendas_cidade,produtos_df,—,withColumn
8,vw_vendas_cidade,pedidos_df,—,filter
9,vw_vendas_cidade,produtos_df,—,filter
